In [1]:
import copy
import csv
import pandas as pd

#### 1) 基準ケース作成

In [2]:
def make_standard_case(standard_case, d, target_keys):
    
    # 基準ケースの作成
    for key in target_keys:
        standard_case[key] = d[key]['typical']
        if 'children' in d[key]:
            if d[key]['children'][d[key]['typical']] != [] :
                for child in d[key]['children'][d[key]['typical']]:
                    standard_case = make_standard_case(standard_case, d, [child])

    return standard_case

#### 2) リスト要素の追加・削除

In [3]:
def add_and_remove_list(list, setting):
    
    # リスト要素の追加
    if 'extra' in setting:
        for element in setting['extra']:
            if element not in list:
                list.append(element)
    
    # リスト要素の削除
    if 'exception' in setting:
        for element in setting['exception']:
            if element in list:
                list.remove(element)
    
    return sorted(list)

#### 3) 要素選択の項目のケース作成

In [4]:
def make_cases_for_options(keys, cases, standard_case, key, d):
    
    # ケース設定用のリスト作成
    temp = [0]
    temp.extend(d[key]['options'])
    temp.extend([max(d[key]['options']) + 1])
    options = add_and_remove_list(temp, d[key])
                     
    # ケース設定
    for option in options:
        case = copy.deepcopy(standard_case)
        case[key] = option
        
        # 条件分岐あり
        if 'children' in d[key]:
            if option in d[key]['children']:
                if d[key]['children'][option] == []:
                    if sum(0 if case[key] == standard_case[key] else 1 for key in keys) == 1 :
                        cases.append(case)
                else:
                    for child in d[key]['children'][option]:
                        cases = make_cases(keys, cases, case, d, [child])
                    
        # 条件分岐なし
        else:
            if sum(0 if case[key] == standard_case[key] else 1 for key in keys) == 1 :
                cases.append(case)
    
    return cases

#### 4) 値入力の項目のケース作成

In [5]:
def make_cases_for_values(keys, cases, standard_case, key, setting):
    
    # ケース設定用のリスト作成
    temp = [
        setting['min'] - setting['dec'],
        setting['min'],
        setting['typical'],
        setting['max'],
        setting['max'] + setting['dec']
    ]
    values = add_and_remove_list(temp, setting)

    # ケース設定
    for value in values:
        case = copy.deepcopy(standard_case)    
        case[key] = value
        if sum(0 if case[key] == standard_case[key] else 1 for key in keys) == 1 :
            cases.append(case)
    
    return cases

#### 5) ケース作成

In [6]:
def make_cases(keys, cases, standard_case, keys_and_settings, target_keys):

    # ケースの生成
    for key in target_keys:
        
        #要素選択の項目
        if keys_and_settings[key]['input_method'] == 'choose_from_options' :
            cases = make_cases_for_options(keys, cases, standard_case, key, keys_and_settings)
        
        #値入力の項目
        if keys_and_settings[key]['input_method'] == 'input_value' :
            cases = make_cases_for_values(keys, cases, standard_case, key, keys_and_settings[key])
        
    return cases

#### 6) csv出力

In [7]:
def output_cases(keys_and_settings):

    # 項目の取得
    keys = [key for key in keys_and_settings]
    target_keys = copy.deepcopy(keys)
    exception_keys = []
    for key in keys_and_settings:
        if 'children' in keys_and_settings[key]:
            for option in keys_and_settings[key]['children'] :
                for element in keys_and_settings[key]['children'][option]:
                    if element not in exception_keys: 
                         exception_keys.append(element)
                            
    # 必須でない項目を除外
    for element in exception_keys:
        if element in target_keys:
            target_keys.remove(element)

    # 基準ケースの作成
    standard_case = {}
    standard_case = make_standard_case(standard_case, keys_and_settings, target_keys)
    
    # テスケースの作成
    cases = [standard_case]
    testcases = make_cases(keys, cases, standard_case, keys_and_settings, target_keys)

    # csv出力
    with open("test_lv1_to_lv2_cases.csv", "w", encoding="Shift_jis") as csv_test_lv1_to_lv2_cases:
        csv_test_lv1_to_lv2_cases.write(','.join(keys) + '\n')

        for testcase in testcases:
            row = [(str(testcase[key]) if key in testcase else '') for key in keys]
            csv_test_lv1_to_lv2_cases.write(','.join(row) + '\n')

#### 7) 項目設定

・床面積のtypicalは、WEBプログラムのデフォルト値を参考に設定。  
・U値のtypicalは、6地域の仕様基準を参考に設定。  
・窓のη値のtypicalは、金属製建具の二層複層ガラスを参考に設定。  
・窓のf値のtypicalは、「当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法」のデフォルト値を使用する場合の6地域を参考に設定。

In [8]:
keys_and_settings = {
    'region'                                 : { 'input_method':'choose_from_options', 'options': [1, 2, 3, 4, 5, 6, 7, 8], 'typical': 1, 'extra':[0, 10] },
    'main_occupant_room_floor_area'          : { 'input_method':'input_value', 'dec':0.01, 'typical': 29.81, 'min':0.01, 'max':999.99, 'extra':[0, 100] },
    'other_occupant_room_floor_area'         : { 'input_method':'input_value', 'dec':0.01, 'typical': 51.34, 'min':0.00, 'max':999.99 },
    'total_floor_area'                       : { 'input_method':'input_value', 'dec':0.01, 'typical':120.08, 'min':0.01, 'max':999.99 },
    'input_method'                           : { 'input_method':'choose_from_options', 'options': [1, 2, 3, 4], 'typical': 1, 'exception':[2, 3, 4] },
    'insulation_type'                        : { 'input_method':'choose_from_options', 'options': [1, 2]   , 'typical': 1,
                                                 'children': {1:['insulation_type_bathroom','u_value_floor_other'], 2:[]}},
    'insulation_type_bathroom'               : { 'input_method':'choose_from_options', 'options': [1, 2, 3], 'typical': 1,
                                                 'children': {1:['u_value_floor_bathroom'], 2:[], 3:[]}},
    'u_value_roof'                           : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.24, 'min':0.00, 'max':10.00 },
    'u_value_wall'                           : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.53, 'min':0.00, 'max':10.00 },
    'u_value_door'                           : { 'input_method':'input_value', 'dec':0.01, 'typical': 4.65, 'min':0.00, 'max':10.00 },
    'u_value_window'                         : { 'input_method':'input_value', 'dec':0.01, 'typical': 4.65, 'min':0.00, 'max':10.00 },    
    'u_value_floor_bathroom'                 : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.48, 'min':0.00, 'max':10.00 },
    'u_value_floor_other'                    : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.48, 'min':0.00, 'max':10.00 },
    'is_psi_value_base_input'                : { 'input_method':'choose_from_options', 'options': [1, 2], 'typical': 1 },
    'psi_value_earthfloor_perimeter_entrance': { 'input_method':'input_value', 'dec':0.01, 'typical': 0.53, 'min':0.00, 'max':10.00 },
    'psi_value_earthfloor_perimeter_bathroom': { 'input_method':'input_value', 'dec':0.01, 'typical': 0.53, 'min':0.00, 'max':10.00 },
    'psi_value_earthfloor_perimeter_other'   : { 'input_method':'input_value', 'dec':0.01, 'typical': 0.53, 'min':0.00, 'max':10.00 },
    'eta_d_value_window_h'                   : { 'input_method':'input_value', 'dec':0.001, 'typical': 0.630, 'min':0.000, 'max':1.000 },
    'eta_d_value_window_c'                   : { 'input_method':'input_value', 'dec':0.001, 'typical': 0.630, 'min':0.000, 'max':1.000 },
    'is_f_value_input'                       : { 'input_method':'choose_from_options', 'options': [1, 2], 'typical': 1,
                                                 'children':{1:['f_value_h', 'f_value_c'], 2:[]}},
    'f_value_h': { 'input_method':'input_value', 'dec':0.001, 'typical': 0.589, 'min':0.000, 'max':1.000 },
    'f_value_c': { 'input_method':'input_value', 'dec':0.001, 'typical': 0.864, 'min':0.000, 'max':1.000 }
}

output_cases(keys_and_settings)

is_f_value_inputがTrueならばf_value_hとf_value_cを入力するというような処理が入る場合  